In [21]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [22]:
df = pd.read_csv('mushrooms.csv')

In [37]:
df.head(20)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


this data is categorical data

In [24]:
df.shape

(8124, 23)

### encode the categorical data into numerical data

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le = LabelEncoder()
#apply transformation on each column
ds = df.apply(le.fit_transform)

In [27]:
ds.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [28]:
# convert it into numpy array
data = ds.values

In [29]:
data.shape

(8124, 23)

In [30]:
type(data)

numpy.ndarray

In [31]:
x_data = data[:,1:]
y_data = data[:,0]

### break the data into train and test

In [32]:
from sklearn.model_selection import train_test_split

In [38]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.2)

In [39]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [40]:
np .unique(y_train)

array([0, 1])

there are 2 types of mushroom

### Building our classisfier 

In [44]:
def prioriprob(y_train,label):
    totalexamples = y_train.shape[0]
    classexamples = np.sum(y_train==label)
    return classexamples/float(totalexamples)

In [45]:
y = np.array([0,5,5,5,0,0,0,1,1,1,1])

In [46]:
prioriprob(y,1)

0.36363636363636365

In [47]:
def conditional_probab(x_train,y_train,feature_col,feature_val,label):
    x_train_filtered = x_train[y_train==label]
    numerator = np.sum(x_train_filtered[:,feature_col]==feature_val)
    denominator = np.sum(y_train==label)
    return numerator/float(denominator)

### compute posterior probability

In [50]:
def prediction(x_train,y_train,x_test):
    classes = np.unique(y_train)
    n_features = x_train.shape[1]
    post_prob = []
    
    for label in classes:
        # post_c = likelihood*prior
        likelihood = 1.0
        for f in range(n_features):
            cond = conditional_probab(x_train,y_train,f,x_test[f],label)
            likelihood*=cond
        priorprobability = prioriprob(y_train,label)
        post = likelihood*priorprobability
        post_prob.append(post)
    pred = np.argmax(post_prob)
    return pred 

In [54]:
output = prediction(x_train,y_train,x_test[5])
print(output)
print(y_train[5])

1
1


In [59]:
def score(x_train,y_train,x_test,y_test):
    pred = []
    for i in range(x_test.shape[0]):
        predict = prediction(x_train,y_train,x_test[i])
        pred.append(predict)
    pred = np.array(pred)
    
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    return accuracy

In [60]:
print(score(x_train,y_train,x_test,y_test))

0.9969230769230769
